In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import math #Math for math.log, math.exp
import pandas as pd #Pandas for dataframes
import seaborn as sns #Seaborn for plots
import matplotlib.pyplot as plt #To configure plot
sns.set_theme(style="darkgrid") #Settings for Seaborn 
sns.set(rc={'figure.figsize':(9,6)})

In [2]:
def A_polynomial(x): #Calculate absorbance coefficient for 1 ppm of A defects at given wavenumber (x)
    absorbance = 0.094038*(0.550912*math.exp(-math.log(2)*((x-1279.86)/17.2349)**2) +
    0.133781*math.exp(-math.log(2)*((x-1219.13)/27.5189)**2) + 
    0.233498*math.exp(-math.log(2)*((x-1305.04)/15.5024)**2) + 
    0.0747244*math.exp(-math.log(2)*((x-1323.17)/9.63831)**2) + 
    0.195502*math.exp(-math.log(2)*((x-1189.32)/61.5864)**2) + 
    0.0190665*math.exp(-math.log(2)*((x-1094.02)/8.05948)**2) + 
    0.0373999*math.exp(-math.log(2)*((x-1178.38)/11.9168)**2) + 
    0.0173979*math.exp(-math.log(2)*((x-1069.84)/153.586)**2) + 
    0.0503451*math.exp(-math.log(2)*((x-1246.31)/9.34388)**2) + 
    0.0447667*math.exp(-math.log(2)*((x-1333.53)/5.59456)**2))
    return absorbance

def B_polynomial(x): #Calculate absorbance coefficient for 1 ppm of B defects at given wavenumber (x)
    absorbance = 0.086879*(0.330958*math.exp(-math.log(2)*((x-1172.84)/22.7015)**2) +
    0.0873933*math.exp(-math.log(2)*((x-1328.94)/6.01119)**2) +
    0.12989*math.exp(-math.log(2)*((x-1209.04)/17.474)**2) +
    0.112641*math.exp(-math.log(2)*((x-1298.06)/24.0692)**2) +
    0.0169314*math.exp(-math.log(2)*((x-1009.77)/-8.17408)**2) +
    0.124944*math.exp(-math.log(2)*((x-1332.31)/2.53534)**2) +
    0.0639029*math.exp(-math.log(2)*((x-1319.52)/11.6064)**2) +
    0.137574*math.exp(-math.log(2)*((x-1249.6)/30.2707)**2) +
    0.163247*math.exp(-math.log(2)*((x-1117.47)/-41.5777)**2) +
    0.100646*math.exp(-math.log(2)*((x-1007.96)/-40.9212)**2) +
    0.036189*math.exp(-math.log(2)*((x-1094.14)/7.13817)**2) +
    0.00649364*math.exp(-math.log(2)*((x-1365.85)/7.24099)**2))
    return absorbance

def C_polynomial(x): #Calculate absorbance coefficient for 1 ppm of C defects at given wavenumber (x)
    absorbance = 0.0648969*(0.298892*math.exp(-math.log(2)*((x-1131.82)/32.6014)**2) +
    0.350026*math.exp(-math.log(2)*((x-1344.41)/0.951707)**2) +
    0.145203*math.exp(-math.log(2)*((x-1279.65)/32.0906)**2) +
    0.142262*math.exp(-math.log(2)*((x-1044.06)/54.839)**2) +
    0.168983*math.exp(-math.log(2)*((x-1128.35)/12.0659)**2) +
    0.0922648*math.exp(-math.log(2)*((x-1309.78)/19.2166)**2) +
    0.282525*math.exp(-math.log(2)*((x-1188.72)/54.2373)**2) +
    0.0695556*math.exp(-math.log(2)*((x-1091.6)/7.72596)**2) +
    -0.0268251*math.exp(-math.log(2)*((x-1336)/2.31933)**2) +
    0.052207*math.exp(-math.log(2)*((x-1328.35)/11.6304)**2) +
    0.0297582*math.exp(-math.log(2)*((x-1346.23)/1.31788)**2) +
    0.017944*math.exp(-math.log(2)*((x-1088.37)/1.71115)**2) +
    0.0387903*math.exp(-math.log(2)*((x-1343.56)/1.91425)**2))
    return absorbance

In [3]:
#Load standard diamond spectrum from csv files, name columns:
standard_diamond = pd.read_csv("HPHT IIa diamond spectrum normalized.dat", sep=" ", names=['w', 'k'],) #Load standard diamond spectrum
#Load set of wavenumbers:
wavenumbers = standard_diamond.iloc[:, 0]
#Calculate defect absorption spectra using polynomials:
A_absorption = pd.DataFrame(data=[[A_polynomial(i)] for i in wavenumbers], columns=["k"]) 
B_absorption = pd.DataFrame(data=[[B_polynomial(i)] for i in wavenumbers], columns=["k"]) 
C_absorption = pd.DataFrame(data=[[C_polynomial(i)] for i in wavenumbers], columns=["k"])


In [4]:
#Create function to plot diamond IR absorption spectrum.
#A,B,C = A,B,C defect concentrations in ppm respectively.
def diamond_spectrum(A, B, C, Full):
    spectrum = pd.DataFrame(data={ #Total spectrum
                #Take wavenumbers from standard diamond spectrum as first (0) column.
                'Wavenumber': standard_diamond.iloc[:, 0],
                #Calculcate absorbance coefficients for each wavenumbers.
                #Total absorbance = diamond_absorbance + A_absorbance + B_absorbance + C_absorbance
                'Absorption coefficient': standard_diamond.iloc[:, 1] + 
                A*A_absorption.iloc[:, 0] + #A defect contribution
                B*B_absorption.iloc[:, 0] + #B defect contribution
                C*C_absorption.iloc[:, 0]}) #C defect contribution
    sns.lineplot(data=spectrum, x="Wavenumber", y="Absorption coefficient", color='b') #Plot with seaborn
    if Full == False: plt.xlim(900,1400) #Set limits if Range is set to 1-phonon
    plt.figure(figsize=(20, 6)) #Set figsize

In [5]:
#Create interactive plot:
interact(diamond_spectrum,
    A=widgets.IntSlider(min=0, max=200, step=5, value=20),
    B=widgets.IntSlider(min=0, max=200, step=5, value=20),
    C=widgets.IntSlider(min=0, max=200, step=5, value=0),
    Full=True)

interactive(children=(IntSlider(value=20, description='A', max=200, step=5), IntSlider(value=20, description='…

<function __main__.diamond_spectrum(A, B, C, Full)>

In [7]:
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__
list(imports())

['builtins',
 'builtins',
 'ipywidgets',
 'math',
 'pandas',
 'seaborn',
 'matplotlib.pyplot',
 'types']